# Stanza

![Latest Version](https://img.shields.io/pypi/v/stanza.svg?colorB=bc4545)
![Python Versions](https://img.shields.io/pypi/pyversions/stanza.svg?colorB=bc4545)

Stanza es un conjunto de herramientas Python NLP que soporta más de 60 idiomas humanos. Está construido con componentes de redes neuronales de alta precisión que permiten un entrenamiento y evaluación eficientes con sus propios datos anotados, y ofrece modelos preentrenados en 100 árboles. Además, Stanza proporciona una interfaz Python estable y oficialmente mantenida para Java Stanford CoreNLP Toolkit.

Vamos a ver cómo ómo configurar Stanza y anotar texto con sus modelos nativos de NLP de redes neuronales.

## 1. Instalación

Stanza sólo es compatible con Python 3.6 y superiores. Instalar e importar Stanza es tan sencillo como ejecutar los siguientes comandos:

In [1]:
!pip install stanza
#conda install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 16.8 MB/s eta 0:00:00


In [2]:
# Carga de la librería
import stanza

## 2. Descarga de los modelos

Podemos descargar modelos con el comando `stanza.download`. El idioma puede especificarse con un nombre completo (por ejemplo, "english") o un código corto (por ejemplo, "en").


Por defecto, los modelos se guardarán en su directorio `~/stanza_resourcePodemos indicar la ruta para guardar los archivos a través del  un argumento `ruta_modelorunt.


In [3]:
# Descarga el modelo inglés en el directorio por defecto
print("Descargando modelo Inglés...")
stanza.download('en')

# Descarga del modelo castellano
# Con verbose=False desactivamos los mensajes
print("Descargando modelo español...")
stanza.download('es', verbose=False)

Descargando modelo Inglés...


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


Descargando modelo español...


### Descarga de modelos preentrenados

Disponibles más de 60 modelos preentrenados -> [Página de modelos](https://stanfordnlp.github.io/stanza/models.html).


## 3. Procesando textos


### Construcción del Pipeline


Para procesamiento de texto, primero debemos unir un `Pipeline` con diferentes unidades `Processor`. El proceso es específico de cada idioma, por lo que debemos especificarlo primero.



- Por defecto, la cadena incluirá todos los procesadores, incluidos la tokenización, la expansión de tokens multipalabra, el etiquetado de partes del discurso, la lematización, el análisis sintáctico de dependencias y el reconocimiento de entidades con nombre (para los idiomas compatibles). Sin embargo, se puede especificar qué proceso queremos incluir con el argumento `proceors`.



- La canalización de Stanza es compatible con CUDA, lo que significa que se utilizará un dispositivo CUDA siempre que esté disponible, de lo contrario se utilizarán CPUs cuando no se encuentre una GPU. Puede forzar el uso de la CPU sin tener en cuenta la GPU configurando `use_g=FPodemos e nuevo, puedes suprimir todos los mensajes impresos configurando `verbose=False`.

Los `Processor`disponibles son:
* **tokenize**: Tokeniza el texto y segmenta las frases.
* **mwt**: Expande los tokens multipalabra (MWT) predichos por el TokenizeProcessor (dámelo = da me lo). Esto sólo es aplicable a algunos idiomas.
* **pos**: Etiqueta los tokens con sus etiquetas POS universales (UPOS), POS específicos del banco de árboles (XPOS) y rasgos morfológicos universales (UFeats). -> https://universaldependencies.org/u/pos/
* **lemma**: Genera los lemas de todas las palabras del documento.
* **depparse**: Proporciona un análisis sintáctico preciso de las dependencias.
* **ner**: Reconocimiento de entidades con nombre para todos los intervalos de tokens del corpus.
* **sentiment**: Asignar puntuaciones de sentimiento por frase (0: Negativo, 1: Neutral, 2:Positivo)
* **constituency**: Analiza cada frase de un documento utilizando un analizador sintáctico de estructura de frases.

`Processor` necesarios para extraer:
* mwt -> tokenize
* pos -> tokenize, mwt
* lemma -> tokenize, mwt, pos
* depparse -> tokenize, mwt, pos, lemma
* ner -> tokenize, mwt
* sentiement -> tokenize, mwt
* constituency -> tokenize, mwt, pos

In [4]:
# Generando el pipeline en inglés con la configuración por defecto
print("Generando el pipeline en inglés...")
en_nlp = stanza.Pipeline('en')

# Generando el pipeline en castellano, usando un 'processor' que obtiene los tokens, lemmas, ... que no registra logs y fuerza el uso de cpu
print("Generando pipeline castellano...")
es_nlp = stanza.Pipeline('es', processors='tokenize,lemma,pos,depparse,sentiment', verbose=False, use_gpu=False)

Generando el pipeline en inglés...
Generando pipeline castellano...


### Anotación de texto
Una vez que una cadena de texto se ha construido correctamente, puede obtener anotaciones de un fragmento de texto simplemente pasando la cadena al objeto de la cadena. La cadena devolverá un objeto `Document`, que puede utilizarse para acceder a anotaciones detalladas. Por ejemplo

In [5]:
# Procesando texto en inglés
en_doc = en_nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
print(type(en_doc))

# Procesando texto en castellano
es_doc = es_nlp("En un lugar de la Mancha de cuyo nombre no quiero acordarme")
print(type(es_doc))

<class 'stanza.models.common.doc.Document'>
<class 'stanza.models.common.doc.Document'>


In [6]:
en_doc

[
  [
    {
      "id": 1,
      "text": "Barack",
      "lemma": "Barack",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 4,
      "deprel": "nsubj:pass",
      "start_char": 0,
      "end_char": 6,
      "ner": "B-PERSON",
      "multi_ner": [
        "B-PERSON"
      ]
    },
    {
      "id": 2,
      "text": "Obama",
      "lemma": "Obama",
      "upos": "PROPN",
      "xpos": "NNP",
      "feats": "Number=Sing",
      "head": 1,
      "deprel": "flat",
      "start_char": 7,
      "end_char": 12,
      "ner": "E-PERSON",
      "multi_ner": [
        "E-PERSON"
      ]
    },
    {
      "id": 3,
      "text": "was",
      "lemma": "be",
      "upos": "AUX",
      "xpos": "VBD",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 4,
      "deprel": "aux:pass",
      "start_char": 13,
      "end_char": 16,
      "ner": "O",
      "multi_ner": [
        "O"
      ]
    },
    {
      "id": 4,
      "text": "b

 Estos campos proporciona información detallada sobre cómo se ha analizado cada token en el texto, incluyendo su categoría gramatical, relaciones con otras palabras, lematización, reconocimiento de entidades... permitiendo realizar un análisis profundo.

In [7]:
es_doc

[
  [
    {
      "id": 1,
      "text": "En",
      "lemma": "en",
      "upos": "ADP",
      "xpos": "sps00",
      "head": 3,
      "deprel": "case",
      "start_char": 0,
      "end_char": 2
    },
    {
      "id": 2,
      "text": "un",
      "lemma": "uno",
      "upos": "DET",
      "xpos": "di0ms0",
      "feats": "Definite=Ind|Gender=Masc|Number=Sing|PronType=Art",
      "head": 3,
      "deprel": "det",
      "start_char": 3,
      "end_char": 5
    },
    {
      "id": 3,
      "text": "lugar",
      "lemma": "lugar",
      "upos": "NOUN",
      "xpos": "ncms000",
      "feats": "Gender=Masc|Number=Sing",
      "head": 0,
      "deprel": "root",
      "start_char": 6,
      "end_char": 11
    },
    {
      "id": 4,
      "text": "de",
      "lemma": "de",
      "upos": "ADP",
      "xpos": "sps00",
      "head": 6,
      "deprel": "case",
      "start_char": 12,
      "end_char": 14
    },
    {
      "id": 5,
      "text": "la",
      "lemma": "el",
      "upos": "DET",


### Más info

Más info sobre cómo construir pipelines -> https://stanfordnlp.github.io/stanfordnlp/pipeline.html.

## 4. Acceso a las anotaciones


Se puede acceder a las anotaciones desde el objeto `Document` devuelto.



Un `Document` contiene una lista de Sentencias. Cada `s,SentenciaSentence` contiene una lista de Palabras  `Word`s. En la mayoría de los casos, las Palabras  `Word`s son equivalentes a los tokens, pero algunos tokens pueden dividirse en múltiples palabras, por ejemplo el token francés `aux` se divide en las palabras `à` y `les`, mientras que en inglés una palabra y un token son equivalentes.



Además, se utiliza un objeto `Span` para representar anotaciones que forman parte de un documento, como las menciones de entidades nombradas.





El siguiente ejemplo itera sobre todas las frases y palabras en inglés, e imprime la información de las mismas una por una:

In [ ]:
for i, sentencia in enumerate(en_doc.sentences):
    print("[Sentencia {}]".format(i+1))
    for palabra in sentencia.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              palabra.text, palabra.lemma, palabra.pos, palabra.head, palabra.deprel))
    print("")

[Sentencia 1]
Barack      	Barack      	PROPN 	4	nsubj:pass  
Obama       	Obama       	PROPN 	1	flat        
was         	be          	AUX   	4	aux:pass    
born        	bear        	VERB  	0	root        
in          	in          	ADP   	6	case        
Hawaii      	Hawaii      	PROPN 	4	obl         
.           	.           	PUNCT 	4	punct       

[Sentencia 2]
He          	he          	PRON  	3	nsubj:pass  
was         	be          	AUX   	3	aux:pass    
elected     	elect       	VERB  	0	root        
president   	president   	NOUN  	3	xcomp       
in          	in          	ADP   	6	case        
2008        	2008        	NUM   	3	obl         
.           	.           	PUNCT 	3	punct       



En un análisis de dependencias, la raíz de la oración es el verbo principal que controla toda la estructura gramatical. La raíz siempre recibe el valor 0.

Iteración sobre todas las menciones de entidades con nombre extraídas e impresión de sus espacios de caracteres y tipos.

In [ ]:
print("Texto mencionado\tTipo\tInicio-Fin")
for ent in en_doc.ents:
    print("{}\t{}\t{}-{}".format(ent.text, ent.type, ent.start_char, ent.end_char))

Texto mencionado	Tipo	Inicio-Fin
Barack Obama	PERSON	0-12
Hawaii	GPE	25-31
2008	DATE	62-66


El algoritmo de entidad nombrada busca secuencias de palabras que frecuentemente representan personas, lugares, organizaciones... Para "Barack Obama", el modelo reconoce estas dos palabras como una entidad de tipo PERSON.

La etiqueta GPE se asigna a los nombres de lugares que están involucrados en un contexto geopolítico. "Hawaii" es reconocido como una entidad geopolítica (GPE), porque es un estado de los Estados Unidos.

En castellano:

In [ ]:
for i, sentencia in enumerate(es_doc.sentences):
    print("[Sentencia {}]".format(i+1))
    for palabra in sentencia.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              palabra.text, palabra.lemma, palabra.pos, palabra.head, palabra.deprel))
    print("Sentimiento de la sentencia: ", sentencia.sentiment)
    print("")

[Sentencia 1]
En          	en          	ADP   	3	case        
un          	uno         	DET   	3	det         
lugar       	lugar       	NOUN  	0	root        
de          	de          	ADP   	6	case        
la          	el          	DET   	6	det         
Mancha      	mancha      	PROPN 	3	nmod        
de          	de          	ADP   	9	case        
cuyo        	cuyo        	DET   	9	det         
nombre      	nombre      	NOUN  	11	obl         
no          	no          	ADV   	11	advmod      
quiero      	querer      	VERB  	3	acl:relcl   
acordar     	acordar     	VERB  	11	xcomp       
me          	yo          	PRON  	12	obj         
Sentimiento de la sentencia:  0



También podemos imprimir directamente un objeto `Word` para ver todas sus anotaciones como un dict de Python:

In [ ]:
word = en_doc.sentences[0].words[0]
print(word)

{
  "id": 1,
  "text": "Barack",
  "lemma": "Barack",
  "upos": "PROPN",
  "xpos": "NNP",
  "feats": "Number=Sing",
  "head": 4,
  "deprel": "nsubj:pass",
  "start_char": 0,
  "end_char": 6
}


In [ ]:
palabra = es_doc.sentences[0].words[0]
print(palabra)

{
  "id": 1,
  "text": "En",
  "lemma": "en",
  "upos": "ADP",
  "xpos": "sps00",
  "head": 3,
  "deprel": "case",
  "start_char": 0,
  "end_char": 2
}


In [ ]:
# Otro ejemplo
es_doc = es_nlp("Me gusta la cerveza fría")


In [ ]:
for i, sentencia in enumerate(es_doc.sentences):
    print("[Sentencia {}]".format(i+1))
    for palabra in sentencia.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              palabra.text, palabra.lemma, palabra.pos, palabra.head, palabra.deprel))
    print("Sentimiento de la sentencia: ", sentencia.sentiment)
    print("")

[Sentencia 1]
Me          	yo          	PRON  	2	obl:arg     
gusta       	gustar      	VERB  	0	root        
la          	el          	DET   	4	det         
cerveza     	cerveza     	NOUN  	2	nsubj       
fría        	frío        	ADJ   	4	amod        
Sentimiento de la sentencia:  0



El verbo "gusta" actúa como la raíz de la oración, siendo el verbo principal que controla toda la estructura. El pronombre "me" se considera un argumento obligatorio del verbo "gusta", funcionando como el receptor de la acción de gustar, es decir, indica quién experimenta la acción (en este caso, el hablante). La palabra "cerveza" es el sujeto de la acción, ya que en la construcción del verbo "gustar" el sujeto es la cosa que gusta, en lugar del receptor de la acción, lo que genera una estructura gramatical diferente a la de otros verbos. "La" es el determinante que acompaña a "cerveza", especificando que se refiere a una cerveza en particular. El adjetivo "fría" describe la cualidad de la cerveza, indicando que es fría. Así, "me" tiene relación con "gusta" porque es el receptor de la acción, y "cerveza" es el sujeto de la acción, mientras que "fría" califica al sustantivo "cerveza".

In [ ]:
es_doc = es_nlp("A pesar de mis esfuerzos, no logré cumplir con las expectativas del proyecto, lo que me dejó sintiéndome frustrado y desanimado.")
for i, sentencia in enumerate(es_doc.sentences):
    print("[Sentencia {}]".format(i+1))
    for palabra in sentencia.words:
        print("{:12s}\t{:12s}\t{:6s}\t{:d}\t{:12s}".format(\
              palabra.text, palabra.lemma, palabra.pos, palabra.head, palabra.deprel))
    print("Sentimiento de la sentencia: ", sentencia.sentiment)
    print("")

[Sentencia 1]
A           	a           	ADP   	5	case        
pesar       	pesar       	NOUN  	1	fixed       
de          	de          	ADP   	1	fixed       
mis         	mi          	DET   	5	det         
esfuerzos   	esfuerzo    	NOUN  	8	obl         
,           	,           	PUNCT 	5	punct       
no          	no          	ADV   	8	advmod      
logré       	lograr      	VERB  	0	root        
cumplir     	cumplir     	VERB  	8	xcomp       
con         	con         	ADP   	12	case        
las         	el          	DET   	12	det         
expectativas	expectativa 	NOUN  	9	obl         
de          	de          	ADP   	15	case        
el          	el          	DET   	15	det         
proyecto    	proyecto    	NOUN  	12	nmod        
,           	,           	PUNCT 	20	punct       
lo          	él          	PRON  	20	det         
que         	que         	PRON  	20	nsubj       
me          	yo          	PRON  	20	obl:arg     
dejó        	dejar       	VERB  	8	advcl       
sintiendo   	sent

### Información extendida

https://stanfordnlp.github.io/stanza/data_objects.html.

## 5. Más info

- [Stanza Homepage](https://stanfordnlp.github.io/stanza/index.html)
- [FAQs](https://stanfordnlp.github.io/stanza/faq.html)
- [GitHub Repo](https://github.com/stanfordnlp/stanza)
- [Reporting Issues](https://github.com/stanfordnlp/stanza/issues)
- [Stanza System Description Paper](http://arxiv.org/abs/2003.07082)
